In [1]:
from promptedgraphs.entity_recognition import extract_entities
from promptedgraphs.data_extraction import extract_data
from promptedgraphs.config import Config

from dotenv import load_dotenv

load_dotenv() 
message = 'Could you suggest some must-visit tourist attractions in Japan during cherry blossom season?'
labels ={
    'travelers': 'A list of who is traveling, their ages, and relationships between each other', 
    'departure_date': 'Departure Date', 
    'return_date': 'Return Date', 
    'expected_length_of_trip': 'Expected Length of Trip', 
    'departure_location': 'Where the travelers are departing from', 
    'budget': 'Any information related to the expected costs or budget limitations for the trip', 
    'trip_reason': 'The purpose of the trip, e.g., fun, work, etc.', 
    'locations': 'A list of all locations mentioned in the request', 
    'accomodations': 'A list of all hotels and other accommodations mentioned in the request', 
    'activities': 'A list of all activities and travel interests mentioned in the request', 
    'needs': 'Accessibility, dietary restrictions, or medical considerations', 
    'accomodation_preferences': 'Preference for types of lodging such as hotels, Airbnbs, or opinions on places to stay', 
    'transportation_preferences': 'Any preference related to how they like to travel between destinations and generally on the trip', 
    'interests': 'Other interests mentioned that the travelers would likely enjoy doing'}

import random
keys = sorted(labels.keys())
random.shuffle(keys)

print("leys", keys)

leys ['return_date', 'departure_date', 'travelers', 'activities', 'accomodations', 'expected_length_of_trip', 'trip_reason', 'interests', 'departure_location', 'budget', 'needs', 'accomodation_preferences', 'transportation_preferences', 'locations']


In [2]:
for i in range(0, len(keys), 5):
    print(i, keys[i:i+5])
    async for state in extract_entities(
                        text=message, 
                        labels={k:v for k,v in labels.items() if k in keys[i:i+5]},
                        name="travel_info",
                        description = "Find closely matching travel information from text",
                        config=Config()):
        print(state)

0 ['return_date', 'departure_date', 'travelers', 'activities', 'accomodations']
5 ['expected_length_of_trip', 'trip_reason', 'interests', 'departure_location', 'budget']
10 ['needs', 'accomodation_preferences', 'transportation_preferences', 'locations']


In [10]:
async for state in extract_entities(
                    text=message, 
                    labels=labels,
                    temperature=0,
                    config=Config()):
    print(state)

EntityReference(start=23, end=53, label='activities', text='must-visit tourist attractions', reason='tourist attractions')
EntityReference(start=57, end=62, label='locations', text='Japan', reason='Japan')
EntityReference(start=70, end=91, label='interests', text='cherry blossom season', reason='cherry blossom season')


In [4]:
async for state in extract_entities(
                    text=message, 
                    labels=labels,
                    model="gpt-4-0613",
                    config=Config()):
    print(state)

EntityReference(start=57, end=62, label='locations', text='Japan', reason='Japan is mentioned as the location where the user wants to visit tourist attractions.')
EntityReference(start=70, end=91, label='interests', text='cherry blossom season', reason="Cherry blossom season is mentioned as the user's interest.")


In [5]:
from pydantic import BaseModel, Field

description = (
    "You are a truthful AI assistant to a luxury travel agent. You only responsed to travel related tasks."
    "Your goal is to work with clients and their help the travel agents plan a great trip."
).strip()

dso_labels = {
    "TRAVELER": "List of travelers with their ages and relationship details.",
    "DATE": "Absolute or relative dates or periods",
    "DEPARTURE LOCATION": "Location from where the travelers will start their journey.",
    "LOCATIONS": "All destinations and places mentioned in the travel request.",
    "ACCOMODATIONS": "Details of hotels and other lodging options mentioned.",
    "ACTIVITIES": "List of activities and attractions highlighted in the travel plan.",
    "NEEDS": "Special requirements such as accessibility, dietary needs, or medical considerations.",
    "INTERESTS": "Additional activities or attractions the travelers might enjoy.",
}


class Traveler(BaseModel):
    name: str = Field(title="Traveler Name")
    age: int | None = Field(None, title="Age of Traveler")
    relationship: str | None = Field(None, title="Relationship with other travelers")


class TravelPlanningState(BaseModel):
    """Defines the data necessary to plan a trip"""

    travelers: list[Traveler] = Field(
        default=[],
        title="TRAVELERS",
        description="A list of who is traveling, their ages, and relationships between each other",
    )
    departure_date: str | None = Field(title="Departure Date")
    return_date: str | None = Field(title="Return Date")
    expected_length_of_trip: str | None = Field(
        default=None, title="Expected Length of Trip"
    )
    departure_location: str | None = Field(
        title="DEPARTURE LOCATION",
        description="Where the travelers are departing from",
    )
    budget: str | None = Field(
        title="BUDGET",
        description="Any information related to the expected costs or budget limitations for the trip",
    )
    trip_reason: str | None = Field(
        title="TRIP REASON",
        description="The purpose of the trip, e.g., fun, work, etc.",
    )
    locations: list[str] | None = Field(
        title="LOCATIONS",
        description="A list of all locations mentioned in the request",
    )
    accomodations: list[str] | None = Field(
        title="ACCOMODATIONS",
        description="A list of all hotels and other accommodations mentioned in the request",
    )
    activities: list[str] | None = Field(
        title="ACTIVITIES",
        description="A list of all activities and travel interests mentioned in the request",
    )
    needs: str | None = Field(
        title="NEEDS",
        description="Accessibility, dietary restrictions, or medical considerations",
    )
    accomodation_preferences: str | None = Field(
        title="ACCOMODATION PREFERENCES",
        description="Preference for types of lodging such as hotels, Airbnbs, or opinions on places to stay",
    )
    transportation_preferences: str | None = Field(
        title="TRANSPORTATION PREFERENCES",
        description="Any preference related to how they like to travel between destinations and generally on the trip",
    )
    interests: list[str] | None = Field(
        default=[],
        title="INTERESTS",
        description="Other interests mentioned that the travelers would likely enjoy doing",
    )


In [11]:
state = None
async for state in extract_data(
    text=message, output_type=TravelPlanningState, config=Config(), 
                    model="gpt-4-0613",force_type=True
):
    print(state)
    break
    s = state.model_dump(
        exclude_defaults=True, exclude_unset=True, exclude_none=True
    )
    for k, v in s.items():
        updated = True
        new_state[k] = v

In [7]:
state

In [8]:
extract_data?

Signature:
extract_data(
    text: str,
    output_type: list[pydantic.main.BaseModel] | pydantic.main.BaseModel,
    config: promptedgraphs.config.Config,
    model='gpt-3.5-turbo-0613',
    temperature=0.0,
    custom_system_message: str | None = None,
    force_type: bool = False,
)
Docstring: <no docstring>
File:      ~/repos/closedloop/PromptedGraphs/promptedgraphs/data_extraction.py
Type:      function

In [17]:
async for state in extract_entities(
                    text=message, 
                    labels={k.lower():'' for k,v in dso_labels.items()},
    temperature=0.0,
                    # model="gpt-4-0613",
                    config=Config()):
    print(state)

In [14]:
The following Text is a question about the NFL. Fill out as many of the below categories as possible from the Text.
If a city is mentioned, use that to determine the NFL team that plays in that city and use that team's mascot for 'Team'.
A week number refers to that week of the NFL season. For example, week 1 games have a 'Min Date' of 2023-09-07 and 'Max Date' of 2023-09-13. When possible, using today's date ({today}) as reference, the Text, and the fact that NFL teams play at most one game a week, determine 'Min Date' and 'Max Date' for those games. 'Max Date' for all games this season is 2024-01-08.
Fill out an integer value for 'Number of Games' if the Text is asking about a certain number of games *for a particular team*. If the Text is asking about ranking games involving many team, DO NOT put a value for Number of Games.
PlayerPosition must be a position on a football team (e.g. running back, tight end, quarterback).

Return a JSON list of lists [entity_type, text] where entity_type MUST be one of the following:
Person-Player, Person-Coach, Team, Place, Min Date, Max Date, Number, PlayerPosition, Odds, Person-Other, Other, Number of Games


{'TRAVELER': 'List of travelers with their ages and relationship details.',
 'DATE': 'Absolute or relative dates or periods',
 'DEPARTURE LOCATION': 'Location from where the travelers will start their journey.',
 'LOCATIONS': 'All destinations and places mentioned in the travel request.',
 'ACCOMODATIONS': 'Details of hotels and other lodging options mentioned.',
 'ACTIVITIES': 'List of activities and attractions highlighted in the travel plan.',
 'NEEDS': 'Special requirements such as accessibility, dietary needs, or medical considerations.',
 'INTERESTS': 'Additional activities or attractions the travelers might enjoy.'}